In [2]:
import ai.koog.prompt.executor.llms.all.simpleOllamaAIExecutor
import ai.koog.prompt.llm.LLMCapability
import ai.koog.prompt.llm.LLMProvider
import ai.koog.prompt.llm.LLModel

val executor = simpleOllamaAIExecutor()
val model = LLModel(
    provider = LLMProvider.Ollama,
    id = "mistral:latest",
    capabilities = listOf(
        LLMCapability.Temperature,
        LLMCapability.Schema.JSON.Simple,
        LLMCapability.Tools
    )
)


In [3]:
import org.example.kagent.createCodingAgentStrategy

val strategy = createCodingAgentStrategy()

In [5]:
import ai.koog.agents.core.agent.config.AIAgentConfig
import ai.koog.prompt.dsl.Prompt

val agentConfig = AIAgentConfig(
    prompt = Prompt.build("coding-assistant") {
        system(
            """
                You are an expert Kotlin coding assistant that can help users with:

                1. **Code Generation**: Create Kotlin code based on requirements
                2. **Test Creation**: Write comprehensive unit tests
                3. **Code Compilation**: Compile Kotlin code and handle errors
                4. **Test Execution**: Run tests and report results
                5. **Project Setup**: Create proper project structures

                ## Workflow:

                1. Understand the user's requirements
                2. Create appropriate project structure if needed
                3. Generate the main code file
                4. Create comprehensive tests
                5. Compile both main code and tests
                6. Run tests and report results
                7. Fix any compilation or test failures

                ## Best Practices:

                - Write clean, idiomatic Kotlin code
                - Include proper error handling
                - Create meaningful test cases
                - Follow Kotlin naming conventions
                - Add appropriate documentation

                ***IMPORTANT*** ALWAYS USE TOOLS TO IMPLEMENT THE TASK!!!
                ***IMPORTANT*** DON'T CHAT WITH ME BEFORE YOU FINISH

                Always explain what you're doing at each step and provide clear feedback about success or failure.
                """.trimIndent()
        )
    },
    model = model,
    maxAgentIterations = 25
)

In [11]:
import ai.koog.agents.core.agent.AIAgent
import ai.koog.agents.features.eventHandler.feature.EventHandler
import kotlin.uuid.ExperimentalUuidApi

@OptIn(ExperimentalUuidApi::class)
val agent = AIAgent(
    promptExecutor = executor,
    strategy = strategy,
    agentConfig = agentConfig,
    installFeatures = {
        install(EventHandler) {
            onBeforeAgentStarted { strategy, _ ->
                println("🚀 Starting coding session with strategy: ${strategy.name}")
            }
            onAgentFinished { strategyName, result ->
                println("✅ Coding session completed: $strategyName")
                println("📋 Final result: $result")
            }
            onAgentRunError { strategyName, uuid, exception
                -> println("🚨 Error occurred for strategy: $strategyName [$uuid]: $exception")
            }
        }
    }
)

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[11], line 3, column 20: This declaration is only available since Kotlin 2.0 and cannot be used with the specified API version 1.9
at Cell In[11], line 5, column 8: This declaration is only available since Kotlin 2.0 and cannot be used with the specified API version 1.9

In [9]:
import kotlinx.coroutines.runBlocking

runBlocking {
    agent.run("write a simple fizzbuzz program")
}